In [12]:
import findspark

In [13]:
findspark.init( '/home/kajili/spark-2.4.5-bin-hadoop2.7' )

In [14]:
import pyspark

In [15]:
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [17]:
df = spark.read.json('./test_data/people.json')

In [18]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [19]:
df.printSchema() #shows columns and their data types

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [10]:
df.columns

['age', 'name']

In [14]:
df.describe()

IllegalArgumentException: 'Unsupported class file major version 55'

### This error occurs because I was using Java 11 with Spark 2.4.5. I followed this StackOverflow link to switch to Java 8:

https://stackoverflow.com/a/56091229:

I ran into this issue when running Jupyter Notebook and Spark using Java 11. I installed and configured for Java 8 using the following steps.

Install Java 8:

```$ sudo apt install openjdk-8-jdk```

Since I had already installed Java 11, I then set my default Java to version 8 using:

```$ sudo update-alternatives --config java```

Select Java 8 and then confirm your changes:

```$ java -version```

Output should be similar to:

```openjdk version "1.8.0_191"
OpenJDK Runtime Environment (build 1.8.0_191-8u191-b12-2ubuntu0.18.04.1-b12)
OpenJDK 64-Bit Server VM (build 25.191-b12, mixed mode)```

I'm now able to run Spark successfully in Jupyter Notebook. The steps above were based on the following guide: https://www.digitalocean.com/community/tutorials/how-to-install-java-with-apt-on-ubuntu-18-04

In [11]:
#trying to run df.describe() again after switching to Java 8

df.describe() #statistical summary of the DataFrame               

DataFrame[summary: string, age: string, name: string]

### Switching to Java 8 did fix the issue with Spark 2.4.5!

In [12]:
#statistical summary of the numeric columns in your DataFrame
df.describe().show() 

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



---



# Defining our own Schema:

In [13]:
# Clarify the Schema specifically

from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [14]:
data_schema = [StructField('age',IntegerType(),True),
               StructField('name',StringType(),True)]

In [15]:
final_struc = StructType(fields=data_schema)

In [16]:
# Create our defined DataFrame using Spark `read` function with our data schema struct as an argument
df2 = spark.read.json('./test_data/people.json', schema=final_struc)

In [17]:
#original automated DataFrame Schema
df.printSchema() 

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [18]:
#our defined DataFrame Schema
df2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



# End of Section 8 Lecture 24

---

<br><br>

# Start of Section 8 Lecture 25:

In [19]:
df2.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [20]:
df2['age']

Column<b'age'>

In [21]:
type(df2['age']) #type is a Column

pyspark.sql.column.Column

---
## Use `select` method to get a dataframe with a single column

In [22]:
df2.select('age')

DataFrame[age: int]

In [24]:
type(df2.select('age')) #type is a DataFrame

pyspark.sql.dataframe.DataFrame

In [25]:
#use .show() to display the actual dataframe column
df2.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



## Use DataFrame `head` function to get rows

In [29]:
df2.head(2)

list

In [30]:
df2.head(2)[0]

Row(age=None, name='Michael')

In [31]:
type(df2.head(2)[0]) # Type is Row

pyspark.sql.types.Row

## Select multiple columns into a DataFrame

In [33]:
df2.select(['age', 'name'])

DataFrame[age: int, name: string]

In [35]:
df2.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Can add a new column using `withColumn` method

In [36]:
df2.withColumn('newage', df2['age'])

DataFrame[age: int, name: string, newage: int]

In [38]:
df2.withColumn('newage', df2['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [39]:
# Can perform operations on whole columns
df2.withColumn('double_age', df2['age'] * 2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [40]:
# Changes not saved, not inplace, have to use a new variable. So df2 was not changed by the `withColumn` function
df2.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [46]:
# saving withColumn operation into df3 variable
df3 = df2.withColumn('double_age', df2['age'] * 2)

In [45]:
df3.show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



## Can rename columns using `withColumnRenamed()` function.

In [47]:
df2.withColumnRenamed('age', 'new_age')

DataFrame[new_age: int, name: string]

In [48]:
df2.withColumnRenamed('age', 'new_age').show()

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



In [50]:
# `withColumnRenamed` also does not perform in place operations, must be saved to a variable as well. 
# df2 was not changed.
df2.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Using Direct SQL to interact with DataFrame

In [51]:
df2.createOrReplaceTempView('people')

In [52]:
results = spark.sql("SELECT * FROM people")

In [53]:
type(results)

pyspark.sql.dataframe.DataFrame

In [54]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [55]:
new_results = spark.sql("SELECT * FROM people WHERE age=30")

In [56]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



# End of Section 8 Lecture 24 & 25: Spark DataFrame Basics